In [1]:
import numpy as np
import cv2

In [2]:
# Parameters for ShiTomasi corner detection (good features to track paper)
corner_track_params = dict(maxCorners = 40, qualityLevel = 0.3, minDistance = 7, blockSize = 7)

In [3]:
# Parameters for lucas kanade optical flow
lk_params = dict(winSize = (200, 200), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [ ]:
# Capture the video

camera = "your_wireless_camera_IP"

cap = cv2.VideoCapture(camera)

# Grabt the very first frame of the stream to track
ret, prev_frame = cap.read()

# convert it into gray scale
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Grab the corners to track
prev_pts = cv2.goodFeaturesToTrack(prev_gray, mask = None, **corner_track_params)
# using that dict here by **

# Create a matching mask of the previouc frame for drawing on later
mask = np.zeros_like(prev_frame)

# run
while True:
    # Grab current frame
    ret, frame = cap.read()

    # Grab gray scale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate the optical flow on the gray scale frame
    next_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prev_pts, None, **lk_params)

    # Using the returned status array (the status output)
    # status output status vector (of unsigned chars); each element of the vector is set to 1 if
    # the flow for the corresponding features has been found, otherwise, it is set to 0.
    good_new = next_pts[status==1]
    good_prev = prev_pts[status==1]

    # Use ravel to get points to draw lines and circles
    for i,(new, prev) in enumerate(zip(good_new, good_prev)):
            # Reshape the array into 1D but keeping the order same
            x_new, y_new = new.ravel()
            x_prev, y_prev = prev.ravel()
            x_new, y_new, x_prev, y_prev = int(x_new), int(y_new), int(x_prev), int(y_prev)

            # Lines will be drawn using the mask created from the first frame
            mask = cv2.line(mask, (x_new,y_new),(x_prev,y_prev), (0,255,0), 3)

            # Draw red circles at corner points
            frame = cv2.circle(frame,(x_new,y_new),5,(0,0,255),-1)

    # Display the image along with the mask we drew the line on
    img = cv2.add(frame, mask)
    frame = cv2.resize(img, (800, 450))
    cv2.imshow("Frame", frame)

    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Now update the previous frame and  previous points
    prev_gray = frame_gray.copy()
    prev_pts = good_new.reshape(-1, 1, 2) # that's how lk works

cv2.destroyAllWindows()
cap.release()